In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
from scipy import signal
from  scipy import ndimage
import matplotlib
from scipy import stats
import ast



In [42]:
df = pd.read_csv('final_exop_data.csv')
# Print the entire DataFrame
print(df)

      Unnamed: 0.1  Unnamed: 0         kepid  \
0                0           0   KIC 1162345   
1                1           1    KIC 757450   
2                2           2    KIC 892667   
3                3           3    KIC 892772   
4                4           4   KIC 1026957   
...            ...         ...           ...   
8881          8881        8885  KIC 12833566   
8882          8882        8886  KIC 12833701   
8883          8883        8887  KIC 12554212   
8884          8884        8888  KIC 12554510   
8885          8885        8889  KIC 12834874   

                                                   time  \
0     [ 1472.11679667  1472.13722971  1472.15766256 ...   
1     [ 1472.11677062  1472.13720366  1472.1576365  ...   
2     [ 1472.11677383  1472.13720687  1472.15763971 ...   
3     [ 1472.11677918  1472.13721222  1472.15764506 ...   
4     [ 1472.11678745  1472.13722049  1472.15765333 ...   
...                                                 ...   
8881  [ 14

In [43]:
filtered_df = df[df['labels'] == 0]

# Get the top 6 rows from the filtered DataFrame
top_6_tuples = filtered_df.head(6)
top_6_tuples.reset_index(drop=True, inplace=True)
# Print the resulting DataFrame
print(top_6_tuples)

   Unnamed: 0.1  Unnamed: 0        kepid  \
0             0           0  KIC 1162345   
1             2           2   KIC 892667   
2             3           3   KIC 892772   
3             4           4  KIC 1026957   
4             5           5  KIC 1160891   
5             6           6  KIC 1162150   

                                                time  \
0  [ 1472.11679667  1472.13722971  1472.15766256 ...   
1  [ 1472.11677383  1472.13720687  1472.15763971 ...   
2  [ 1472.11677918  1472.13721222  1472.15764506 ...   
3  [ 1472.11678745  1472.13722049  1472.15765333 ...   
4  [ 1472.11678409  1472.13721714  1472.15764999 ...   
5  [ 1472.11679794  1472.13723099  1472.15766383 ...   

                                                flux  labels  
0  [ 290573.1875   290615.40625  290711.09375 ......       0  
1  [ 76636.7421875  76644.0390625  76627.0859375 ...       0  
2  [ 12183.046875    12195.359375    12198.278320...       0  
3  [ 119163.3046875  119160.125      119167.23

In [4]:
# for i in range(6):
#     x=filtered_df['flux'].iloc[i][1:-1].split(',')
#     # Convert the list of strings to a list of integers
#     x= [float(item) for item in x]
#     y=ast.literal_eval(filtered_df['time'].iloc[i])
#     plt.plot(y,x)
#     plt.show()
    

In [5]:
# filtered_df_1 = df[df['labels'] == 1]

# # Get the top 6 rows from the filtered DataFrame
# top_6_tuples_1 = filtered_df_1.head(6)
# top_6_tuples_1.reset_index(drop=True, inplace=True)
# # Print the resulting DataFrame
# print(top_6_tuples_1)

In [6]:
# for i in range(6):
#     x=filtered_df_1['flux'].iloc[i][1:-1].split(',')
#     # Convert the list of strings to a list of integers
#     x= [float(item) for item in x]
#     y=ast.literal_eval(filtered_df_1['time'].iloc[i])
#     plt.scatter(y,x,s=0.1)
#     plt.show()

In [7]:
# for i in range(6):
#     x=filtered_df_1['flux'].iloc[i][1:-1].split(',')
#     x= [float(item) for item in x]
#     flux2 = ndimage.gaussian_filter(x, sigma=1)
    
#     time =ast.literal_eval(filtered_df_1['time'].iloc[i])
#     plt.figure(figsize=(15,5))
#     plt.ylabel('Flux, e-/s')
#     plt.xlabel('Time, hours')
#     plt.plot(time, flux2)
#     print(flux2)

In [8]:
#for i in range(6):
#    x=filtered_df_1['flux'].iloc[i][1:-1].split(',')
#    x= [float(item) for item in x]
#    flux2 = ndimage.gaussian_filter(x, sigma=0.1)
#    flux2=flux2.rolling(window=window_size, min_periods=1, center=True).mean()
#    time =ast.literal_eval(filtered_df_1['time'].iloc[i])
#    plt.figure(figsize=(15,5))
#    plt.ylabel('Flux, e-/s')
#    plt.xlabel('Time, hours')
#    plt.plot(time, flux3)
#    print(flux3)
    

In [44]:
for kepid in top_6_tuples['kepid']:
    print(kepid)

KIC 1162345
KIC 892667
KIC 892772
KIC 1026957
KIC 1160891
KIC 1162150


In [45]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
#import tensorflow as tf
from light_curve import binning
#from light_curve import kepler_io
from light_curve import util
# from tf_util import example_util
from third_party.kepler_spline import kepler_spline



In [46]:
def process_light_curve(all_time, all_flux):
  """Removes low-frequency variability from a light curve.

  Args:
    all_time: A list of numpy arrays; the time values of the raw light curve.
    all_flux: A list of numpy arrays corresponding to the time arrays in
      all_time.

  Returns:
    time: 1D NumPy array; the time values of the light curve.
    flux: 1D NumPy array; the normalized flux values of the light curve.
  """
  # Split on gaps.
  all_time, all_flux = util.split(all_time, all_flux, gap_width=0.75)

  # Fit a piecewise-cubic spline with default arguments.
  spline = kepler_spline.fit_kepler_spline(all_time, all_flux, verbose=False)[0]

  # Concatenate the piecewise light curve and spline.
  time = np.concatenate(all_time)
  flux = np.concatenate(all_flux)
  spline = np.concatenate(spline)

  # In rare cases the piecewise spline contains NaNs in places the spline could
  # not be fit. We can't normalize those points if the spline isn't defined
  # there. Instead we just remove them.
  finite_i = np.isfinite(spline)
  if not np.all(finite_i):
    time = time[finite_i]
    flux = flux[finite_i]
    spline = spline[finite_i]

  # "Flatten" the light curve (remove low-frequency variability) by dividing by
  # the spline.
  flux /= spline

  return time, flux

In [47]:
def fit_kepler_spline(all_time,
                      all_flux,
                      bkspace_min=0.5,
                      bkspace_max=20,
                      bkspace_num=20,
                      maxiter=5,
                      penalty_coeff=1.0,
                      verbose=True):
  """Fits a Kepler spline with logarithmically-sampled breakpoint spacings.

  Args:
    all_time: List of 1D numpy arrays; the time values of the light curve.
    all_flux: List of 1D numpy arrays; the flux values of the light curve.
    bkspace_min: Minimum breakpoint spacing to try.
    bkspace_max: Maximum breakpoint spacing to try.
    bkspace_num: Number of breakpoint spacings to try.
    maxiter: Maximum number of attempts to fit each spline after removing badly
      fit points.
    penalty_coeff: Coefficient of the penalty term for using more parameters in
      the Bayesian Information Criterion. Decreasing this value will allow more
      parameters to be used (i.e. smaller break-point spacing), and vice-versa.
    verbose: Whether to log individual spline errors. Note that if bkspaces
      contains many values (particularly small ones) then this may cause logging
      pollution if calling this function for many light curves.

  Returns:
    spline: List of numpy arrays; values of the best-fit spline corresponding to
        to the input flux arrays.
    metadata: Object containing metadata about the spline fit.
  """
  # Logarithmically sample bkspace_num candidate break point spacings between
  # bkspace_min and bkspace_max.
  bkspaces = np.logspace(
      np.log10(bkspace_min), np.log10(bkspace_max), num=bkspace_num)

  return choose_kepler_spline(
      all_time,
      all_flux,
      bkspaces,
      maxiter=maxiter,
      penalty_coeff=penalty_coeff,
      verbose=verbose)


In [48]:
def choose_kepler_spline(all_time,
                         all_flux,
                         bkspaces,
                         maxiter=5,
                         penalty_coeff=1.0,
                         verbose=True):
  """Computes the best-fit Kepler spline across a break-point spacings.

  Some Kepler light curves have low-frequency variability, while others have
  very high-frequency variability (e.g. due to rapid rotation). Therefore, it is
  suboptimal to use the same break-point spacing for every star. This function
  computes the best-fit spline by fitting splines with different break-point
  spacings, calculating the Bayesian Information Criterion (BIC) for each
  spline, and choosing the break-point spacing that minimizes the BIC.

  This function assumes a piecewise light curve, that is, a light curve that is
  divided into different segments (e.g. split by quarter breaks or gaps in the
  in the data). A separate spline is fit for each segment.

  Args:
    all_time: List of 1D numpy arrays; the time values of the light curve.
    all_flux: List of 1D numpy arrays; the flux values of the light curve.
    bkspaces: List of break-point spacings to try.
    maxiter: Maximum number of attempts to fit each spline after removing badly
      fit points.
    penalty_coeff: Coefficient of the penalty term for using more parameters in
      the Bayesian Information Criterion. Decreasing this value will allow more
      parameters to be used (i.e. smaller break-point spacing), and vice-versa.
    verbose: Whether to log individual spline errors. Note that if bkspaces
      contains many values (particularly small ones) then this may cause logging
      pollution if calling this function for many light curves.

  Returns:
    spline: List of numpy arrays; values of the best-fit spline corresponding to
        to the input flux arrays.
    metadata: Object containing metadata about the spline fit.
  """
  # Initialize outputs.
  best_spline = None
  metadata = SplineMetadata()

  # Compute the assumed standard deviation of Gaussian white noise about the
  # spline model. We assume that each flux value f[i] is a Gaussian random
  # variable f[i] ~ N(s[i], sigma^2), where s is the value of the true spline
  # model and sigma is the constant standard deviation for all flux values.
  # Moreover, we assume that s[i] ~= s[i+1]. Therefore,
  # (f[i+1] - f[i]) / sqrt(2) ~ N(0, sigma^2).
  scaled_diffs = [np.diff(f) / np.sqrt(2) for f in all_flux]
  scaled_diffs = np.concatenate(scaled_diffs) if scaled_diffs else np.array([])
  if not scaled_diffs.size:
    best_spline = [np.array([np.nan] * len(f)) for f in all_flux]
    metadata.light_curve_mask = [
        np.zeros_like(f, dtype=bool) for f in all_flux
    ]
    return best_spline, metadata

  # Compute the median absolute deviation as a robust estimate of sigma. The
  # conversion factor of 1.48 takes the median absolute deviation to the
  # standard deviation of a normal distribution. See, e.g.
  # https://www.mathworks.com/help/stats/mad.html.
  sigma = np.median(np.abs(scaled_diffs)) * 1.48

  for bkspace in bkspaces:
    nparams = 0  # Total number of free parameters in the piecewise spline.
    npoints = 0  # Total number of data points used to fit the piecewise spline.
    ssr = 0  # Sum of squared residuals between the model and the spline.

    spline = []
    light_curve_mask = []
    bad_bkspace = False  # Indicates that the current bkspace should be skipped.
    for time, flux in zip(all_time, all_flux):
      # Fit B-spline to this light-curve segment.
      try:
        spline_piece, mask = kepler_spline(
            time, flux, bkspace=bkspace, maxiter=maxiter)
      except InsufficientPointsError as e:
        # It's expected to occasionally see intervals with insufficient points,
        # especially if periodic signals have been removed from the light curve.
        # Skip this interval, but continue fitting the spline.
        if verbose:
          warnings.warn(str(e))
        spline.append(np.array([np.nan] * len(flux)))
        light_curve_mask.append(np.zeros_like(flux, dtype=bool))
        continue
      except SplineError as e:
        # It's expected to get a SplineError occasionally for small values of
        # bkspace. Skip this bkspace.
        if verbose:
          warnings.warn("Bad bkspace {}: {}".format(bkspace, e))
        metadata.bad_bkspaces.append(bkspace)
        bad_bkspace = True
        break

      spline.append(spline_piece)
      light_curve_mask.append(mask)

      # Accumulate the number of free parameters.
      total_time = np.max(time) - np.min(time)
      nknots = int(total_time / bkspace) + 1  # From the bspline implementation.
      nparams += nknots + 3 - 1  # number of knots + degree of spline - 1

      # Accumulate the number of points and the squared residuals.
      npoints += np.sum(mask)
      ssr += np.sum((flux[mask] - spline_piece[mask])**2)

    if bad_bkspace or not npoints:
      continue

    # The following term is -2*ln(L), where L is the likelihood of the data
    # given the model, under the assumption that the model errors are iid
    # Gaussian with mean 0 and standard deviation sigma.
    likelihood_term = npoints * np.log(2 * np.pi * sigma**2) + ssr / sigma**2

    # Penalty term for the number of parameters used to fit the model.
    penalty_term = nparams * np.log(npoints)

    # Bayesian information criterion.
    bic = likelihood_term + penalty_coeff * penalty_term

    if best_spline is None or bic < metadata.bic:
      best_spline = spline
      metadata.light_curve_mask = light_curve_mask
      metadata.bkspace = bkspace
      metadata.likelihood_term = likelihood_term
      metadata.penalty_term = penalty_term
      metadata.bic = bic

  if best_spline is None:
    # All bkspaces resulted in a SplineError, or all light curve intervals had
    # insufficient points.
    best_spline = [np.array([np.nan] * len(f)) for f in all_flux]
    metadata.light_curve_mask = [
        np.zeros_like(f, dtype=bool) for f in all_flux
    ]

  return best_spline, metadata
